In [ ]:
from ome_zarr_models.v04 import Image
from ome_zarr_models import open_ome_zarr
from pathlib import Path
import zarr

path = Path("/media/alessandro/T7/tibiae/pyramidal_chunked_tibia_1")
assert path.exists()
zarr_group = zarr.open(path, mode="r+")
ome_zarr_group = open_ome_zarr(zarr_group)
print(f"Type of the dataset: {type(ome_zarr_group)}")
print(f"OME-Zarr version: {ome_zarr_group.ome_zarr_version}")

In [ ]:
ome_zarr_image = Image.from_zarr(zarr_group)

print(ome_zarr_image.attributes)
metadata = ome_zarr_image.attributes
from rich.pretty import pprint
pprint(metadata)
pprint(metadata.multiscales[0].datasets)

zarr_array = zarr_group[metadata.multiscales[0].datasets[0].path]
pprint(zarr_array)



iterate chunk-wise


In [ ]:
zarr_arr = zarr_group[metadata.multiscales[0].datasets[0].path]
chunk_shape = zarr_arr.chunks
shape = zarr_arr.shape

colour=1
for i in range(0, shape[0], chunk_shape[0]):
    for j in range(0, shape[1], chunk_shape[1]):
        for k in range(0, shape[2], chunk_shape[2]):
            chunk_idx = (
                i // chunk_shape[0] * (shape[1] // chunk_shape[1]) * (shape[2] // chunk_shape[2]) +
                j // chunk_shape[1] * (shape[2] // chunk_shape[2]) +
                k // chunk_shape[2]
            )
            if True or chunk_idx % 2 == 1:
                chunk = zarr_arr[
                    i:i+chunk_shape[0],
                    j:j+chunk_shape[1],
                    k:k+chunk_shape[2]
                ]
                chunk = (chunk > 50) * colour
                zarr_arr[
                    i:i+chunk_shape[0],
                    j:j+chunk_shape[1],
                    k:k+chunk_shape[2]    
                ] = chunk
                colour = (colour+1)%256
                if colour == 0:
                    colour = colour+1
                print(f"Thresholded chunk at ({i}, {j}, {k})")
                print(f"colour: {colour}")

for d in [0,1,2]:
    level_d_array = zarr_group[ome_zarr_image.attributes.multiscales[0].datasets[d].path]
    level_d_array[:] = zarr_arr[::2**d, ::2**d, ::2**d]

In [ ]:
import napari

viewer = napari.Viewer()
viewer.open(path, plugin="napari-ome-zarr")
napari.run()

Right-click on the layer, and hit convert to Labels.